In [3]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import zipfile

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda')


def load_data_jay_lyrics():
    """加载歌词数据集"""
    with zipfile.ZipFile('../data/jaychou/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')                             # corpus_chars 以字符形式读入数据
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')           # 将其中的换行符换为空格
    corpus_chars = corpus_chars[0:10000]                                        # 取前10000个字符作为训练的数据
    idx_to_char = list(set(corpus_chars))                                       # idx_to_char 按照字典顺序排列的汉字列表 如：{'去', '化', ' ', '和', '要',}
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])       # char_to_idx "汉字": "对应的索引" 形式的字典 如：{'去': 0, '化': 1, ' ': 2, '和': 3, '要': 4, '这': 5}
    vocab_size = len(char_to_idx)                                               # vocab_size 词典中不同字符的个数
    corpus_indices = [char_to_idx[char] for char in corpus_chars]               # corpus_indices 歌词顺序的每个字符对应字典中的索引
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

In [4]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))

    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数

    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])


will use cuda


### 定义模型

In [5]:
def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), 
            torch.zeros((batch_size, num_hiddens), device=device))


In [6]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)


In [7]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)


epoch 40, perplexity 213.886308, time 0.39 sec
 - 分开 我不的我 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 
 - 不分开 我不的我 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 
epoch 80, perplexity 67.544547, time 0.41 sec
 - 分开 我想你这 我不要 我不要 我不要 我不要 我不要 我不不 我不不 我不不觉 我不不觉 我不不这 我
 - 不分开 我想你这 我不要 我不要 我不要 我不要 我不要 我不不 我不不 我不不觉 我不不觉 我不不这 我
epoch 120, perplexity 15.253453, time 0.40 sec
 - 分开 我想我 你你 我想 这样 我想要再牵着你 是情开的手快默一找 我想揍你已经很久 别想躲 说你眼睛看
 - 不分开 我不要这样 我不要这样 我不的话生活 我爱你 别怪我 我想就这样着着你的手 不放开开开简不不 不要
epoch 160, perplexity 4.191525, time 0.39 sec
 - 分开 你已我 你爱我 一九怎么 一九 在什么 印地的明在 干在在兮 快使用双截棍 哼哼哈兮 快使用双截棍
 - 不分开你是我不想你你 我不要再样 我不要再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已来不


### 简洁实现

In [8]:
lr = 1e-2 # 注意调整学习率
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(lstm_layer, vocab_size)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)


epoch 40, perplexity 1.020728, time 0.15 sec
 - 分开 我有你看棒球 想这样没担忧 唱着歌 一直走 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害
 - 不分开 我有一双翅膀 二双翅膀 随时出发 偷偷出发 我一定带我妈走  从前的教育别人的家庭  别人的爸爸种
epoch 80, perplexity 1.012660, time 0.15 sec
 - 分开 陷入了危险边缘Baby  我的世界已狂风暴雨 Wu  爱情来的太快就像龙卷风 离不开暴风圈来不及逃
 - 不分开  你在的黑色幽默 说散 你想很久了吧? 我的认真败给黑色幽默 走过了很多地方 我来到伊斯坦堡 就像
epoch 120, perplexity 1.009428, time 0.15 sec
 - 分开 陷入了危险边缘Baby  我的世界已狂风暴雨 Wu  爱情来的太快就像龙卷风 离不开暴风圈来不及逃
 - 不分开  你叫我爸爸爸爸爸和汉堡 我想要你的微笑每天都能看到  我知道这里很美但家乡的你更美走过了很多地方
epoch 160, perplexity 1.009951, time 0.15 sec
 - 分开的玩笑 想通 却又再考倒我 说散 你想很久了吧? 败给你的黑色幽默 说散 你想很久了吧? 我的认真败
 - 不分开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样没担忧 唱着歌 一直走 我想
